


# 主要内容
好习惯，数据文件夹分为input和output

注input文件里放下面这个链接的比赛下载的数据分别为`train_set.csv`和`test_set.csv`

[「二分类算法」提供银行精准营销解决方案 ](https://www.kesci.com/home/competition/5c234c6626ba91002bfdfdd3) 案例
- 特征工程前的准备 (拼接测试训练集)
- 数据探索Exploratory Data Analysis，简称EDA
- 字符型特征转数字(LabelEncoder)
- lightgbm baseline模型
- kfold

In [7]:
# 从sklearn 调入所需要的包
# from sklearn import datasets
from sklearn.model_selection import train_test_split #数据分隔出训练集和验证集
import lightgbm as lgb
import numpy as np
import pandas as pd
#导入精度和召回
from sklearn.metrics import precision_score, recall_score
import lightgbm as lgb
import os

In [8]:
path = os.getcwd()
train = pd.read_csv(os.path.join(path, 'input', 'train_set.csv'))
test = pd.read_csv(os.path.join(path, 'input', 'test_set.csv'))
# print(train.info())
# train.describe()
# train.corr()
# train.job.value_counts()

## 特征工程前拼接训练测试集

In [9]:
# data=pd.concat([train,test])  这个方法可能会出bug
test['y']=-1
data = train.append(test).reset_index(drop=True)
# data = pd.concat([train,test]).reset_index(drop=True)
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


In [10]:
import os
import json
import gc
# os.system('pip install numba')
from numba import jit
#tqdm
# os.system('pip install tqdm')
from tqdm import tqdm_notebook
from tqdm import tqdm

#Integrated model
# os.system('pip install lightgbm')
import lightgbm as lgb
# os.system('pip install catboost==0.15.2')
# import catboost as cbt
# os.system('pip install xgboost')
# import xgboost as xgb

#base import 
import numpy as np
import pandas as pd

# about sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler as std
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import f1_score
#about time
import time
import datetime 
from datetime import datetime, timedelta
#Garbage collection
import gc
# scipy
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
import scipy.spatial.distance as dist
#other
from collections import Counter 
from statistics import mode 
    #warning
import warnings
warnings.filterwarnings("ignore")
import json 
import math
from itertools import product
import ast 

## 字符串转数字

In [11]:
cat_col = [i for i in data.select_dtypes(object).columns if i not in ['ID','y']]
for i in tqdm_notebook(cat_col):
    lbl = LabelEncoder()
#     data['count_' + i] = data.groupby([i])[i].transform('count')
    data[i] = lbl.fit_transform(data[i].astype(str))

In [12]:
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,4,1,2,0,291,1,0,2,9,8,150,2,-1,0,3,0
1,2,42,9,0,0,0,5076,1,0,0,7,0,99,1,251,2,1,0
2,3,47,0,1,1,0,104,1,1,0,14,5,77,2,-1,0,3,0
3,4,28,4,2,1,0,-994,1,1,0,18,5,174,2,-1,0,3,0
4,5,42,9,0,1,0,2974,1,0,2,21,8,187,5,-1,0,3,0


1ID Int客户唯一标识

2age Int客户年龄

3job String客户的职业

4marital String婚姻状况

5education String受教育水平

6default String是否有违约记录

7balance Int每年账户的平均余额

8housing String是否有住房贷款

9loan String是否有个人贷款

10contact String与客户联系的沟通方式

11day Int最后一次联系的时间（几号）

12month String最后一次联系的时间（月份）

13duration Int最后一次联系的交流时长

14campaign Int在本次活动中，与该客户交流过的次数

15pdays Int距离上次活动最后一次联系该客户，过去了多久（999表示没有联系过）16previousInt在本次活动之前，与该客户交流过的次数

17poutcome String上一次活动的结果

18y Int预测客户是否会订购定期存款业务

In [17]:
features = [i for i in data.columns if i not in ['ID','y']]
print(features, len(features))

['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome'] 16


In [18]:
import lightgbm as lgb
model = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=30, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=1500, objective='binary',metric= 'auc',
    subsample=0.95, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.02, random_state=2017
    )
data1 =data[data['y']!=-1][features]
label1 =data[data['y']!=-1]['y']
testx= data[data['y']==-1][features]

train_x, test_x, train_y, test_y = train_test_split(data1, label1, test_size=0.3, random_state=42)
model.fit(train_x,train_y)
test_pre = model.predict_proba(testx)[:,1]#, num_iteration=model.best_iteration

In [20]:
model.predict_proba(testx)

array([[0.97314761, 0.02685239],
       [0.99782871, 0.00217129],
       [0.99755609, 0.00244391],
       ...,
       [0.99878009, 0.00121991],
       [0.99880483, 0.00119517],
       [0.98294149, 0.01705851]])

In [21]:
print(len(test_pre))

10852


In [22]:
pre=data[data['y']==-1][['ID']]
pre['pred']=test_pre
print(len(pre))

10852


In [24]:
pre.head()

,ID,pred
25317,25318,0.026852
25318,25319,0.002171
25319,25320,0.002444
25320,25321,0.882458
25321,25322,0.029544


In [26]:
#5折交叉验证
from sklearn.model_selection import KFold,StratifiedKFold
n_splits=5
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)


train_x = data[data['y']!=-1][features]
train_y = data[data['y']!=-1]['y']


res=data[data['y']==-1][['ID']]
res['pred'] = 0


test_x= data[data['y']==-1][features]

for train_idx, val_idx in kfold.split(train_x):
    model.random_state = model.random_state + 1
    train_x1 = train_x.loc[train_idx]
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x.loc[val_idx]
    test_y1 = train_y.loc[val_idx]
    #,(vali_x,vali_y)
    model.fit(train_x1, 
              train_y1,
              eval_set=[(train_x1, train_y1),(test_x1, test_y1)],
              eval_metric='auc',
              early_stopping_rounds=100)
    
    res['pred'] += model.predict_proba(test_x)[:,1]

res['pred'] = res['pred']/n_splits

[1]	training's auc: 0.769551	valid_1's auc: 0.785035
Training until validation scores don't improve for 100 rounds.
[2]	training's auc: 0.909669	valid_1's auc: 0.905679
[3]	training's auc: 0.921179	valid_1's auc: 0.915109
[4]	training's auc: 0.922362	valid_1's auc: 0.917034
[5]	training's auc: 0.92218	valid_1's auc: 0.916793
[6]	training's auc: 0.921917	valid_1's auc: 0.916965
[7]	training's auc: 0.924098	valid_1's auc: 0.91946
[8]	training's auc: 0.924703	valid_1's auc: 0.91992
[9]	training's auc: 0.925889	valid_1's auc: 0.920906
[10]	training's auc: 0.92641	valid_1's auc: 0.921389
[11]	training's auc: 0.926683	valid_1's auc: 0.921823
[12]	training's auc: 0.927474	valid_1's auc: 0.922501
[13]	training's auc: 0.928514	valid_1's auc: 0.924048
[14]	training's auc: 0.928733	valid_1's auc: 0.923858
[15]	training's auc: 0.929077	valid_1's auc: 0.924346
[16]	training's auc: 0.929165	valid_1's auc: 0.924128
[17]	training's auc: 0.929757	valid_1's auc: 0.924016
[18]	training's auc: 0.93126	val

[300]	training's auc: 0.960907	valid_1's auc: 0.935435
[301]	training's auc: 0.960961	valid_1's auc: 0.93547
[302]	training's auc: 0.961034	valid_1's auc: 0.935443
[303]	training's auc: 0.961113	valid_1's auc: 0.935477
[304]	training's auc: 0.961162	valid_1's auc: 0.935512
[305]	training's auc: 0.961247	valid_1's auc: 0.935541
[306]	training's auc: 0.961302	valid_1's auc: 0.935556
[307]	training's auc: 0.961368	valid_1's auc: 0.935566
[308]	training's auc: 0.961457	valid_1's auc: 0.93559
[309]	training's auc: 0.961549	valid_1's auc: 0.935603
[310]	training's auc: 0.9616	valid_1's auc: 0.9356
[311]	training's auc: 0.961683	valid_1's auc: 0.935616
[312]	training's auc: 0.961734	valid_1's auc: 0.935625
[313]	training's auc: 0.961788	valid_1's auc: 0.935608
[314]	training's auc: 0.961843	valid_1's auc: 0.935605
[315]	training's auc: 0.961922	valid_1's auc: 0.93562
[316]	training's auc: 0.961999	valid_1's auc: 0.935629
[317]	training's auc: 0.962058	valid_1's auc: 0.935662
[318]	training's 

[20]	training's auc: 0.928841	valid_1's auc: 0.917766
[21]	training's auc: 0.929045	valid_1's auc: 0.917772
[22]	training's auc: 0.929371	valid_1's auc: 0.918313
[23]	training's auc: 0.929851	valid_1's auc: 0.918424
[24]	training's auc: 0.929879	valid_1's auc: 0.918193
[25]	training's auc: 0.929988	valid_1's auc: 0.91807
[26]	training's auc: 0.930541	valid_1's auc: 0.919114
[27]	training's auc: 0.930629	valid_1's auc: 0.918898
[28]	training's auc: 0.930891	valid_1's auc: 0.919051
[29]	training's auc: 0.930943	valid_1's auc: 0.919027
[30]	training's auc: 0.931245	valid_1's auc: 0.919009
[31]	training's auc: 0.931219	valid_1's auc: 0.918946
[32]	training's auc: 0.931451	valid_1's auc: 0.919396
[33]	training's auc: 0.931646	valid_1's auc: 0.919367
[34]	training's auc: 0.931499	valid_1's auc: 0.919268
[35]	training's auc: 0.931558	valid_1's auc: 0.919258
[36]	training's auc: 0.931601	valid_1's auc: 0.91935
[37]	training's auc: 0.931659	valid_1's auc: 0.919219
[38]	training's auc: 0.931604	

[320]	training's auc: 0.963233	valid_1's auc: 0.931541
[321]	training's auc: 0.963298	valid_1's auc: 0.931549
[322]	training's auc: 0.96339	valid_1's auc: 0.931521
[323]	training's auc: 0.963432	valid_1's auc: 0.93153
[324]	training's auc: 0.963495	valid_1's auc: 0.931546
[325]	training's auc: 0.963582	valid_1's auc: 0.931552
[326]	training's auc: 0.963668	valid_1's auc: 0.931587
[327]	training's auc: 0.963705	valid_1's auc: 0.931601
[328]	training's auc: 0.963803	valid_1's auc: 0.931638
[329]	training's auc: 0.963858	valid_1's auc: 0.931658
[330]	training's auc: 0.963889	valid_1's auc: 0.931648
[331]	training's auc: 0.96395	valid_1's auc: 0.931658
[332]	training's auc: 0.964045	valid_1's auc: 0.931736
[333]	training's auc: 0.964118	valid_1's auc: 0.931716
[334]	training's auc: 0.96415	valid_1's auc: 0.931687
[335]	training's auc: 0.964227	valid_1's auc: 0.931696
[336]	training's auc: 0.964293	valid_1's auc: 0.93174
[337]	training's auc: 0.964345	valid_1's auc: 0.931726
[338]	training'

[137]	training's auc: 0.945003	valid_1's auc: 0.933668
[138]	training's auc: 0.945069	valid_1's auc: 0.933703
[139]	training's auc: 0.945152	valid_1's auc: 0.933732
[140]	training's auc: 0.945293	valid_1's auc: 0.93386
[141]	training's auc: 0.945473	valid_1's auc: 0.933953
[142]	training's auc: 0.945614	valid_1's auc: 0.933992
[143]	training's auc: 0.945643	valid_1's auc: 0.933889
[144]	training's auc: 0.945715	valid_1's auc: 0.933845
[145]	training's auc: 0.945824	valid_1's auc: 0.9339
[146]	training's auc: 0.945931	valid_1's auc: 0.934001
[147]	training's auc: 0.94605	valid_1's auc: 0.934233
[148]	training's auc: 0.946128	valid_1's auc: 0.93426
[149]	training's auc: 0.94621	valid_1's auc: 0.934278
[150]	training's auc: 0.946339	valid_1's auc: 0.934375
[151]	training's auc: 0.946368	valid_1's auc: 0.934275
[152]	training's auc: 0.946408	valid_1's auc: 0.934215
[153]	training's auc: 0.946511	valid_1's auc: 0.934257
[154]	training's auc: 0.946636	valid_1's auc: 0.934442
[155]	training's

...

[72]	training's auc: 0.93884	valid_1's auc: 0.930017
[73]	training's auc: 0.938926	valid_1's auc: 0.930001
[74]	training's auc: 0.939021	valid_1's auc: 0.929952
[75]	training's auc: 0.93907	valid_1's auc: 0.929863
[76]	training's auc: 0.939335	valid_1's auc: 0.930205
[77]	training's auc: 0.939635	valid_1's auc: 0.930494
[78]	training's auc: 0.939777	valid_1's auc: 0.930679
[79]	training's auc: 0.939865	valid_1's auc: 0.930721
[80]	training's auc: 0.940049	valid_1's auc: 0.930906
[81]	training's auc: 0.940172	valid_1's auc: 0.931065
[82]	training's auc: 0.940228	valid_1's auc: 0.930981
[83]	training's auc: 0.940265	valid_1's auc: 0.930919
[84]	training's auc: 0.940299	valid_1's auc: 0.930843
[85]	training's auc: 0.940478	valid_1's auc: 0.931037
[86]	training's auc: 0.940742	valid_1's auc: 0.931282
[87]	training's auc: 0.940782	valid_1's auc: 0.931317
[88]	training's auc: 0.940834	valid_1's auc: 0.931252
[89]	training's auc: 0.940933	valid_1's auc: 0.931274
[90]	training's auc: 0.941006	

[370]	training's auc: 0.965014	valid_1's auc: 0.939364
[371]	training's auc: 0.965091	valid_1's auc: 0.939345
[372]	training's auc: 0.96518	valid_1's auc: 0.939338
[373]	training's auc: 0.965254	valid_1's auc: 0.939341
[374]	training's auc: 0.965319	valid_1's auc: 0.939381
[375]	training's auc: 0.965366	valid_1's auc: 0.939374
[376]	training's auc: 0.965425	valid_1's auc: 0.939341
[377]	training's auc: 0.965487	valid_1's auc: 0.939355
[378]	training's auc: 0.965539	valid_1's auc: 0.939358
[379]	training's auc: 0.965613	valid_1's auc: 0.939357
[380]	training's auc: 0.96568	valid_1's auc: 0.939327
[381]	training's auc: 0.96575	valid_1's auc: 0.93931
[382]	training's auc: 0.965837	valid_1's auc: 0.939309
[383]	training's auc: 0.965886	valid_1's auc: 0.939329
[384]	training's auc: 0.965926	valid_1's auc: 0.939312
[385]	training's auc: 0.966002	valid_1's auc: 0.939295
[386]	training's auc: 0.966063	valid_1's auc: 0.939296
[387]	training's auc: 0.966113	valid_1's auc: 0.93927
[388]	training'

[668]	training's auc: 0.978913	valid_1's auc: 0.940027
[669]	training's auc: 0.978966	valid_1's auc: 0.940057
[670]	training's auc: 0.979017	valid_1's auc: 0.940085
[671]	training's auc: 0.979047	valid_1's auc: 0.940088
[672]	training's auc: 0.979077	valid_1's auc: 0.940081
[673]	training's auc: 0.979116	valid_1's auc: 0.940102
[674]	training's auc: 0.979144	valid_1's auc: 0.94008
[675]	training's auc: 0.979164	valid_1's auc: 0.940081
[676]	training's auc: 0.979193	valid_1's auc: 0.940091
[677]	training's auc: 0.979229	valid_1's auc: 0.940098
[678]	training's auc: 0.97925	valid_1's auc: 0.940094
[679]	training's auc: 0.979284	valid_1's auc: 0.940099
[680]	training's auc: 0.979315	valid_1's auc: 0.940091
[681]	training's auc: 0.979345	valid_1's auc: 0.940109
[682]	training's auc: 0.979382	valid_1's auc: 0.940092
[683]	training's auc: 0.979421	valid_1's auc: 0.940068
[684]	training's auc: 0.97946	valid_1's auc: 0.94007
[685]	training's auc: 0.979494	valid_1's auc: 0.940062
[686]	training

[256]	training's auc: 0.959068	valid_1's auc: 0.928423
[257]	training's auc: 0.959121	valid_1's auc: 0.928467
[258]	training's auc: 0.959184	valid_1's auc: 0.928462
[259]	training's auc: 0.959273	valid_1's auc: 0.928425
[260]	training's auc: 0.959322	valid_1's auc: 0.928413
[261]	training's auc: 0.959393	valid_1's auc: 0.928364
[262]	training's auc: 0.959522	valid_1's auc: 0.928497
[263]	training's auc: 0.959597	valid_1's auc: 0.92847
[264]	training's auc: 0.959705	valid_1's auc: 0.92854
[265]	training's auc: 0.959751	valid_1's auc: 0.928531
[266]	training's auc: 0.959801	valid_1's auc: 0.928554
[267]	training's auc: 0.959935	valid_1's auc: 0.928657
[268]	training's auc: 0.960025	valid_1's auc: 0.928682
[269]	training's auc: 0.960082	valid_1's auc: 0.928669
[270]	training's auc: 0.96015	valid_1's auc: 0.928667
[271]	training's auc: 0.960222	valid_1's auc: 0.928703
[272]	training's auc: 0.960269	valid_1's auc: 0.92872
[273]	training's auc: 0.960335	valid_1's auc: 0.928728
[274]	training

[554]	training's auc: 0.975449	valid_1's auc: 0.929995
[555]	training's auc: 0.975487	valid_1's auc: 0.929962
[556]	training's auc: 0.975522	valid_1's auc: 0.929976
[557]	training's auc: 0.975563	valid_1's auc: 0.929969
[558]	training's auc: 0.975612	valid_1's auc: 0.929959
[559]	training's auc: 0.975664	valid_1's auc: 0.929981
[560]	training's auc: 0.975699	valid_1's auc: 0.929975
[561]	training's auc: 0.975756	valid_1's auc: 0.929983
[562]	training's auc: 0.975815	valid_1's auc: 0.929995
[563]	training's auc: 0.975849	valid_1's auc: 0.930015
[564]	training's auc: 0.975904	valid_1's auc: 0.930002
[565]	training's auc: 0.975942	valid_1's auc: 0.929988
[566]	training's auc: 0.975992	valid_1's auc: 0.929949
[567]	training's auc: 0.976038	valid_1's auc: 0.929937
[568]	training's auc: 0.976062	valid_1's auc: 0.929953
[569]	training's auc: 0.976109	valid_1's auc: 0.929993
[570]	training's auc: 0.976142	valid_1's auc: 0.929979
[571]	training's auc: 0.976178	valid_1's auc: 0.930003
[572]	trai

In [29]:
res.to_csv(os.path.join(path, 'output', 'submission5_fold.csv'), index=False)